<a href="https://colab.research.google.com/github/vivivicdjdk/machine/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#트리의 앙상블

**정형 데이터(structed data) vs 비정형 데이터(unstructed data)**

정형 데이터: CSV 파일 처럼, 구조를 가지고 있는 데이터

비정형 데이터: 텍스트 데이터, 사진, 디지털 음악 등 구조를 가지고 있지 않은 데이터 <- 6장

---

지금까지 배운 머신러닝 알고리즘은 정형 데이터에 적합
* k-최근접 이웃, 선형 회귀, 릿지, 라쏘, 다항 회귀, 로지스틱 회귀, 결정 트리

##앙상블 학습(ensemble learning)

정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘

대다수의 앙상블 학습 알고리즘은 결정 트리를 기반으로 함

##랜덤 포레스트(random forest)

앙상블 학습 중 가장 유명하고 안정적인 성능을 제공

여러개의 결정 트리를 랜덤하게 만들고 각 결정 트리의 예측을 사용해 최종 예측을 출력

어떻게 결정 트리를 랜덤하게 만들 것인가가 핵심

  1. 훈련하기 위한 데이터를 랜덤하게 만듦
      * 입력한 훈련 데이터에서 랜덤하게 샘플을 추출함
      * 중복된 샘플을 추출할 수 있음
      * 이렇게 만들어진 샘플을 **부트스트랩 샘플(bootstrap sample)**이라 함
      * 기본적으로 훈련 세트의 크기와 동일하게 설정됨
        * 예) 1000개의 샘플로 구성된 훈련 세트가 주어지면 그 훈련 세트로부터 랜덤하게 1000개의 샘플을 중복이 가능하도록 선택

  2. 부트스트랩 샘플로 결정 트리를 훈련

    * 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음
    * RandomForestClassifier 분류 모델은 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택
    * RandomForestRegressor 회귀 모델은 전체 특성을 사용
  
  3. 정해진 수 만큼 위 과정을 반복

  

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 훈련

분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 결정

회귀일 때는 각 트리의 예측을 평균함

랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음

랜덤 포레스트는 누락된 값이 있어도 처리가 가능하다는 장점이 있음

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42) # n_jobs: 모든 CPU코어 사용
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)  # n_jobs: 최대한 병렬로 교차 검증 수행
                                                                                            # return_train_score: 훈련 세트 점수도 반환, 과대적합을 파악하는데 용이
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


과대적합임

이 예제는 매우 간단하고 특성이 많지 않아서 그리드 서치를

사용하더라도 하이퍼파라미터 튜닝의 결과가 크게 나아지지 않음

---

특성 중요도 출력

In [2]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


5-1의 중요도와 비교하면, 두 번째 특성의 중요도가 낮아지고 나머지 두 특성의 중요도가 조금씩 상승

랜덤포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문에

하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음

따라서 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 됨

**oob_score**

랜덤하게 부트스트랩 샘플을 선택하기 때문에 사용할 수 있는 RandomForestClassifier에서 제공하는 재미있는 기능이 있음

***OOB(out of bag) 샘플**

부트스트랩 샘플에 포함되지않는 샘플

이 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있음

In [3]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


교차검증에서 얻은 점수와 매우 비슷한 결과를 얻음

따라서 OOB점수를 사용하면 교차 검증을 대신할 수 있어 훈련 세트에 더 많은 샘플을 사용할 수 있음

##엑스트라트리(extra trees)

랜덤 포레스트와 비슷하게 동작

* 부트스트랩 샘플을 사용하지 않고, 각 결정 트리를 만들 때 전체 훈련 세트를 사용

* 노드를 분할할 때 가장 좋은 분할을 찾지 않고 랜덤으로 분할함

엑스트라 트리에서 사용하는 결정 트리는 splitter='random'인 결정 트리

* DecisionTreeClassifier(splitter='random')

* 그러나 완전히 동일하지는 않음

하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블하기 때문에 과대 적합을 막고 검증 세트의 점수를 높이는 효과가 있음

In [4]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


랜덤 포레스트와 비슷한 결과를 얻음

이 예제는 특성이 많지 않아 두 모델의 차이가 크지 않음

보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야함

그러나 랜덤하게 노드를 분할하기 때문에 계산 속도가 빠름

In [5]:
et.fit(train_input, train_target)
print(et.score(train_input, train_target))
print(et.score(test_input, test_target))
print(et.feature_importances_)


0.996921300750433
0.8861538461538462
[0.20183568 0.52242907 0.27573525]


## Bagging vs Boosting

위 두 방법은 배깅(Bagging) 방법

여러 개의 모델을 랜덤하게 만들고 각 모델의 결과를 종합하여 결과를 출력

아래 방법들은 부스팅(Bossting) 방식

모델을 순차적으로 여러개 만들되 이전 모델을 보완한 새로운 모델을 만들고 최종적으로 생성된 모든 모델을 하나로 합침

##그레이디언트 부스팅(gradient boosting)

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식

사이킷런에서는 기본적으로 깊이가 3인 결정 트리 100개를 사용

깊이가 얕은 결정 트리를 사용하므로 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음

---

그레이디언트란 이름이 붙은 이유는 경사 하강법을 사용하여 트리를 앙상블에 추가하기 때문

분류에서는 로지스틱 손실 함수를 사용

회귀에서는 평균 제곱 오차 함수를 사용

---

경사 하강법이 손실 함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정이라면, 그레이디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동

경사 하강법에서 손실 함수의 낮은 곳으로 천천히 조금씩 이동해야하는 것처럼 그레이디언트 부스팅도 마찬가지이므로 깊이가 얕은 트리를 사용

학습률 매개변수로 속도를 조절

예측에 사용될 데이터가 많은 경우에 사용

##쉽게 이해해보자

부스팅이므로 예측 모델을 순차적으로 더해나간 후 최종모델을 예측

즉, 이전 모델의 오류(실제값과 예측값의 차이)를 새로운 모델을 추가하면서 줄이는 것

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
gb.fit(train_input, train_target)
print(gb.score(train_input, train_target))
print(gb.score(test_input, test_target))

0.8881086892152563 0.8720430147331015
0.8880123147969983
0.8669230769230769


그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강함

학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음

학습률(learning_rate)은 0.1이 기본값

트리의 개수(n_estimators)는 100이 기본값

In [7]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


과대적합을 적절히 억제하고 있음

In [12]:
gb.fit(train_input, train_target)
print(gb.score(train_input, train_target))
print(gb.score(test_input, test_target))
print(gb.feature_importances_)

0.9382335963055609
0.8707692307692307
[0.15872278 0.68010884 0.16116839]


일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있음

그러나 순서대로 트리를 추가하기 때문에 훈련 속도가 느림

즉, n_jobs 매개변수가 없음

**subsample**

재미있는 매개변수

트리 훈련에 사용할 훈련 세트의 비율을 정하며 기본값은 1.0

1보다 작으면 훈련 세트의 일부를 사용함

이러한 경우, 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하여 진행하는 확률적 경사 하강하강법이나 미니배치 경사 하강법과 비슷

##히스토그램 기반 부스팅(histogram_based gradient boosting)

정형 데이터를 다루는 머신러닝 알고리즘 중에서 가장 인기가 높은 알고리즘

입력 특성은 256개의 구간으로 나누므로 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음

256개의 구간 중에서 하나를 떼어놓고 누락된 값을 위해서 사용

즉, 어떤 샘플에 누락된 값이 있다면 떼어놓은 구간의 값을 대신 사용

따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없음

---

히스토그램 : 데이터를 일정한 구간으로 쪼개서 막대그래프로 표현

입력 데이터의 각 특성을 256개의 구간으로 나누어 256개의 값으로 변환

예를 들어, 입력 데이터는 하나의 특성만을 가지고 있고, 특성에 1부터 512까지의 수가 분포되어 있다고 하자

그렇다면 1,2를 1로 3,4를 2로 치환하는 식으로 특성의 범위를 256개의 구간으로 축소할 수 있음

따라서 최적의 분할을 찾기 위해서 각 특성당 256번만 계한을 하면 되므로 효율적

특히 실수 범위를 정수로 범위를 축소할 수 있다는 점이 뛰어남

---

max_iter: 부스팅 반복 횟수(n_estimaters 대신 사용). 성능을 높이려면 이 매개변수를 조절

In [8]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대적합을 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능을 제공

히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산해보자

permitation_imporances()

* 특성을 하나씩 랜덤하게 섞어서 모델의 성능 변화를 관찰하여 어떤 특성이 중요한지 계산함

  * 예를 들어, 3가지 특성이 있다면
  * 각 특성별로 값들을 랜덤하게 섞어 3가지 데이터에 대해 모델을 평가
  * 원 데이터의 모델 평가값과 각 데이터별 모델 평가값의 차이를 출력

* 사이킥런에서 제공하는 추정기 모델에 사용 가능

*중요도(importances), 평균(importances_mean), 표준편차(importances_std)를 출력

n_repeats: 랜덤하게 섞을 횟수, 기본값은 5

특성의 개수 *n_repats만큼 모델 평가를 수행

___

훈련 세트에 대해서도 특성 중요도를 계산할 수 있음

In [9]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


테스트 세트에 대한 특성 중요도 계산

In [10]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [11]:
hgb.score(test_input, test_target)

0.8723076923076923